# Cálculo explícito de los polinomios de Hermite utilizando la RR3T
**Juan Antonio Villegas Recio**
> Nota: Este Notebook debe ser ejecutado utilizando un 'kernel' de SageMath, preferiblemente con una versión igual o superior a la 9.3.

El objetivo es hallar una sucesión de polinomios ortogonales mónicos de Hermite utilizando `SageMath`, haciendo algunas mediciones de tiempos de ejecución.

Primero de todo, importamos funciones para realizar integrales y creamos `P`, que es el equivalente al anillo de polinomios con coeficientes racionales $\mathbb Q[x]$.

In [1]:
from sage.symbolic.integration.integral import integrate
P.<x> = PolynomialRing(QQ)

Los polinomios de Hermite son ortogonales respecto a la función peso $\rho(x) = e^{-x^2}$, $\forall x\in\mathbb R$. Esta función peso puede dar lugar bien al producto escalar 
$$\left\langle f,g\right\rangle=\int_{-\infty}^{+\infty}f(x)g(x) e^{-x^2}dx$$
o bien al funcional lineal
$$\mathcal{L}[f] = \int_{-\infty}^{+\infty} f(x) e^{-x^2}dx.$$
Creamos funciones para ambos casos, aunque de acuerdo a la RR3T utilizaremos únicamente la correspondiente al funcional lineal.

In [2]:
def dot_H(f,g):
    return integrate(f*g*e**(-x**2), x, -infinity, infinity)

def L(f):
    return integrate(f*e**(-x**2), x, -infinity, infinity)

Finalmente creamos una función que, dado $n$, calcule el $n$-ésimo polinomio de Hermite utilizando la RR3T.

In [3]:
def RR3T_H(n):
    if n == -1:
        return P(0)
    elif n == 0:
        return P(1)
    else:
        P_1 = RR3T_H(n-1)
        P_2 = RR3T_H(n-2)
        beta = L(x*P_1**2)/L(P_1**2)
        if P_2 == P(0):
            return P((x-beta)*P_1)
        else:
            gamma = L(x*P_1*P_2)/L(P_2**2)
            return P((x-beta)*P_1 - gamma*P_2)

Véamos entonces los $11$ primeros polinomios de Hermite:

In [4]:
Npol = 11
for i in range(Npol):
    print(str(i) + ": " + str(RR3T_H(i)))

0: 1
1: x
2: x^2 - 1/2
3: x^3 - 3/2*x
4: x^4 - 3*x^2 + 3/4
5: x^5 - 5*x^3 + 15/4*x
6: x^6 - 15/2*x^4 + 45/4*x^2 - 15/8
7: x^7 - 21/2*x^5 + 105/4*x^3 - 105/8*x
8: x^8 - 14*x^6 + 105/2*x^4 - 105/2*x^2 + 105/16
9: x^9 - 18*x^7 + 189/2*x^5 - 315/2*x^3 + 945/16*x
10: x^10 - 45/2*x^8 + 315/2*x^6 - 1575/4*x^4 + 4725/16*x^2 - 945/32


Con el objetivo de hacer un análisis empírico de la eficiencia de este método, haremos $10$ ejecuciones independientes del cálculo de cada polinomio y, mediante la media de los $10$ tiempos de ejecución encontraremos una estimación bastante acertada del tiempo medio de ejecución del cálculo de cada polinomio.

In [5]:
import time
Nejecuciones = 10
tiempos_calculo = [[] for _ in range(Npol)]
for i in range(Nejecuciones):
    for j in range(Npol):
        start_time = time.time()
        RR3T_H(j)
        tiempos_calculo[j].append(time.time() - start_time)

for i in range(Npol):
    print(str(i) + ": " + str(sum(tiempos_calculo[i])/Nejecuciones))

0: 1.4853477478027343e-05
1: 0.014149808883666992
2: 0.038803386688232425
3: 0.09683995246887207
4: 0.1985522747039795
5: 0.35666980743408205
6: 0.633054780960083
7: 1.074570655822754
8: 1.8145904779434203
9: 2.984163427352905
10: 4.941139125823975


Si en lugar de $10$ se desea realizar un número distinto de ejecuciones basta con cambiar el valor de la variable `Nejecuciones`, y si en lugar de calcular los $11$ primeros polinomios de Hermite se desean calcular más o menos se sugiere modificar la variable `Npol`.

Vemos en este caso con las 10 ejecuciones de los 11 primeros polinomios de Hermite que el tiempo de ejecución crece rápidamente con $n$. Probemos cuánto tardaría el programa en calcular $P_{15}$.

In [6]:
start_time = time.time()
RR3T_H(15)
time.time() - start_time

55.52601671218872

Cerca de un minuto, por lo que este método no escala demasiado bien.